In [1]:
def randomprime(i):
    return random_prime(2**i-1,True,2**(i-1))

In [2]:
l = 2048

q = randomprime(l)
p = randomprime(l)

n = p * q
phi = (p-1)*(q-1)

e = ZZ.random_element(phi)
R = IntegerModRing(n)

while gcd(e, phi) != 1:
    e = ZZ.random_element(phi)

bezout = xgcd(e,phi)
d = Integer(mod(bezout[1],phi));

In [3]:
# Public key
(n,e)

(657263651769519555596735757999573600127094327884561335759180057198010693513698581339091176655535081205255362759430136052210698424245714652251510562754849724102258537485207346808361074749237810560893892734210680523279146500696644584138873151550540750966287468678051905369820349155416799847568431271378213108419400204856620578518621208709878996613851262827693888944727419431689636749514207919760286097888378366859225512427690240834365994517476676556995190034303669987554581020711109371827479129856974130407836328803891425994640210228370940203345685624415891107287259306972512664770488400935465093920419228244524759288936010876038606350859226611953943519609052433333530545883758206262601840131223458543965094586585156639108812783797624710585051799755511610600901041204536880759247731260043558257420225923420133991038523074701676486419897537821052769430778685725982441835070834941182801223415021832308444134738055983015043802989145883893550001331357272298599386849839200342304198092251216241296905429306

In [4]:
# Private key
(p,q,d)

(24401343359202846372692459515667450983238829494276531482345909556567032047079961875333245676809516249532507683471600162831527431424798575123431439324409170606688486571045489777501933515028891535959518812331047535977264864726327178652079427570107441411341936582490790928739834543081487198082457826842444193091716557205765483101368425683780396562910567878549540530961893288537433264230043566778589021205010969937243763432644320308357566269185404720507006417568461264707964685342808905988768247870178437765998347957514026709903818984095474962817919753447400246192801628122945547793069743024521018474908682823168362683771,
 2693555195278360807998678629388399296010483673592908846042917756531565204077054017394092951185642659026359566175512497872468916898452842536630445265198547216416852104489883952486825753975879564265857599918724139994696915566884291560442664258392960710468985129960786719540626137949801159631668947425401197683132522037012264187849838807946966550010586991107787596635778527857469919

In [5]:
import hashlib
import binascii

from sage.crypto.util import bin_to_ascii
from random import SystemRandom

nBits = 1024
k0BitsInt = 256
k0BitsFill = '0256b'
encoding = 'utf-8'

oracle1 = hashlib.sha256()
oracle2 = hashlib.sha256()

def CharsToBinary(msg):
    bits = bin(int(binascii.hexlify(msg), 16))
    return bits

def BinaryToChars(bits):
    r = bin_to_ascii(bits)
    return r
    
def paddingOAEP(msg):
  
    randBitStr = format(SystemRandom().getrandbits(k0BitsInt), k0BitsFill)
    binMsg = CharsToBinary(msg)
    
    if len(binMsg) <= (nBits-k0BitsInt):
        k1Bits = nBits - k0BitsInt - len(binMsg) 
        zeroPaddedMsg = binMsg + ('0'*k1Bits)
    
    oracle1.update(randBitStr.encode(encoding))
    x = format(int(zeroPaddedMsg, 2) ^^ int(oracle1.hexdigest(), 16), '0768b')
    oracle2.update(x.encode(encoding))
    y = format(int(randBitStr, 2) ^^ int(oracle2.hexdigest(), 16), k0BitsFill)

    return x+y

def unpad(msg):

    x = msg[0:768]
    y = msg[768:]

    oracle2.update(x.encode(encoding))
    r = format(int(y,2) ^^ int(oracle2.hexdigest(), 16), k0BitsFill)

    oracle1.update(r.encode(encoding))
    msgWith0s = format(int(x,2) ^^ int(oracle1.hexdigest(), 16), '0768b')

    msg = BinaryToChars(msgWith0s)
    return msg

In [6]:
def encrypt(msg):
    msgWithPad = paddingOAEP(msg)
    m = '0b' + msgWithPad
    zz = ZZ(m)
    a = R(zz)
    ct = a**e
    return ct

def decpryt(ct):
    b = R(ct)
    dm = b**d
    zz = ZZ(dm)
    inteiro = Integer(zz)
    m1 = inteiro.binary()
    m = '00'+ m1
    msg = unpad(m)
    return msg

In [7]:
from random import randint

msg = "Hello world"

ct = encrypt(msg)
m = decpryt(ct)

print(msg == m)

False
